## Module基类常用方法

> add_module(name, module)

将子模块添加到当前模块。

> register_module()

add_module方法的封装，用于将新的`name:module`键值对加入module中。

> apply(fn)

+ 递归地将函数应用于所有子模块。
+ apply方法可以用于任何submodule（通过.children()或者self.获取到的）
+ 常用来初始化模型参数（同torch.nn.init）

example:


In [47]:
import torch
@torch.no_grad()  # 不计算梯度，不反向传播
def init_weights(m):
    print(m)
    if type(m) == nn.Linear:
        m.weight.fill_(1.0)
        print(m.weight)
net = nn.Sequential(nn.Linear(2, 2), nn.Linear(2, 2),nn.ReLU(),nn.Sequential(nn.Linear(2, 2),nn.ReLU()))
net.apply(init_weights)

Linear(in_features=2, out_features=2, bias=True)
Parameter containing:
tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
Linear(in_features=2, out_features=2, bias=True)
Parameter containing:
tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
ReLU()
Linear(in_features=2, out_features=2, bias=True)
Parameter containing:
tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
ReLU()
Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): ReLU()
)
Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
  (2): ReLU()
  (3): Sequential(
    (0): Linear(in_features=2, out_features=2, bias=True)
    (1): ReLU()
  )
)


Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
  (2): ReLU()
  (3): Sequential(
    (0): Linear(in_features=2, out_features=2, bias=True)
    (1): ReLU()
  )
)

> buffers(recurse=True)

模型中需要保存下来的参数包括两种：

+ 一种是反向传播需要更新：parameter，可以通过parameter()返回
+ 一种是反向传播不需要更新的：buffer，可以通过buffer()返回



> children()：
返回网络模型里的组成元素的迭代器。类似`modules()`方法。二者对比可参考：[https://blog.csdn.net/u013066730/article/details/94600978](https://blog.csdn.net/u013066730/article/details/94600978)

> register_buffer(name: str, tensor: Optional[Tensor], persistent: bool = True)

在当前模块中添加一个buffer变量，例如，现在需要手写一个BatchNorm，那么其`running_mean`并不是一个parameter，这就需要用下述方式注册一个buffer：

```python
class BatchNorm(nn.Module):
    def __init__(self,..):
        self.register_buffer('running_mean',torch.zeros(num_features))
        self.register_buffer('running_variance',torch.ones(num_features))
```

> register_parameter(self, name: str, param: Optional[Parameter])

用于在当前模块中添加一个parameter变量，其中参数param是一个Parameter类型（继承至tensor类型，nn.parameter.Parameter）。

Example:

In [45]:
import torch
import torch.nn as nn

class GaussianModel(nn.Module):

    def __init__(self):
        super(GaussianModel, self).__init__()

        self.register_parameter('mean', nn.Parameter(torch.zeros(1),
                                                     requires_grad=True))
        
        self.pdf = torch.distributions.Normal(self.state_dict()['mean'],
                                              torch.tensor([1.0]))
    def forward(self, x):
        return -self.pdf.log_prob(x)

model = GaussianModel()
for name, param in model.named_parameters():
    print(name,param.size())

mean torch.Size([1])


In [36]:
for i in net.children():
    print(i)

Linear(in_features=2, out_features=2, bias=True)
Linear(in_features=2, out_features=2, bias=True)
ReLU()
Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): ReLU()
)


> cpu() / cuda()

将模型的parameters和buffers移动到CPU/GPU

> eval()

将module设置为验证模式，会影响一些特定modules，如：Dropout，BatchNorm等

> load_state_dict(state_dict, strict=True)

将 state_dict 中的参数(parameters)和缓冲区(buffers)复制到此模块及其子模块中。如果 strict 为 True，则 state_dict 的键必须与该模块的 state_dict() 函数返回的键完全匹配。

> named_buffers(prefix='', recurse=True)

返回module buffers' name的迭代器，example： 

In [37]:
for name, buf in net.named_buffers():
   if name in ['running_var']:
       print(buf.size())

> named_children()

返回直接子模块的迭代器，产生模块的名称以及模块本身。

In [38]:
for name, module in net.named_children():
    # if name in ['conv4', 'conv5']:
    print(name,module)

0 Linear(in_features=2, out_features=2, bias=True)
1 Linear(in_features=2, out_features=2, bias=True)
2 ReLU()
3 Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): ReLU()
)


> named_modules

返回网络中所有模块的迭代器，产生模块的名称以及模块本身。

In [39]:
for name, module in net.named_modules():
    print(name, module)

 Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
  (2): ReLU()
  (3): Sequential(
    (0): Linear(in_features=2, out_features=2, bias=True)
    (1): ReLU()
  )
)
0 Linear(in_features=2, out_features=2, bias=True)
1 Linear(in_features=2, out_features=2, bias=True)
2 ReLU()
3 Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): ReLU()
)
3.0 Linear(in_features=2, out_features=2, bias=True)
3.1 ReLU()


> named_parameters(prefix='', recurse=True)

返回模块参数的迭代器，产生参数的名称以及参数本身。

In [42]:
for name, param in net.named_parameters():
    print(name,param.size())

0.weight torch.Size([2, 2])
0.bias torch.Size([2])
1.weight torch.Size([2, 2])
1.bias torch.Size([2])
3.0.weight torch.Size([2, 2])
3.0.bias torch.Size([2])


> get_submodule(target: str) -> 'Module'

从Module中获取子module，example：

In [51]:
net.get_submodule('3.0')  # 获取第0个子模块的内嵌滴0个子模块

Linear(in_features=2, out_features=2, bias=True)

> state_dict()

返回包含模块整个状态的字典。 包括参数和持久缓冲区（例如运行平均值）。键是对应的参数和缓冲区名称。不包括设置为 None 的参数和缓冲区。常用于保存模型参数。

保存模型例子：

```python
# Additional information
EPOCH = 5
PATH = "model.pt"
LOSS = 0.4

torch.save({
            'epoch': EPOCH,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': LOSS,
            }, PATH)
```

加载模型例子：
```python
model = Net()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
# - or -
model.train()
```
更多详情参考：[https://pytorch.org/tutorials/recipes/recipes/saving_and_loading_a_general_checkpoint.html](https://pytorch.org/tutorials/recipes/recipes/saving_and_loading_a_general_checkpoint.html)

> get_parameter(target: str)

根据参数名得到参数，exp：

In [55]:
net.get_parameter('1.weight')

Parameter containing:
tensor([[1., 1.],
        [1., 1.]], requires_grad=True)

> get_buffer(target: str)

根据buffer名得到buffer值，用法同get_parameter。

> _apply(fn)

+ 对所有的module、parameter、buffer都进行一个fn

Example：.cpu / .cuda()源码

```python
class Module:
    def cuda(self: T, device: Optional[Union[int, device]] = None) -> T:
        r"""Moves all model parameters and buffers to the GPU.

        This also makes associated parameters and buffers different objects. So
        it should be called before constructing optimizer if the module will
        live on GPU while being optimized.

        .. note::
            This method modifies the module in-place.

        Args:
            device (int, optional): if specified, all parameters will be
                copied to that device

        Returns:
            Module: self
        """
        return self._apply(lambda t: t.cuda(device))
```

> type(dst_type)

将所有的parameters和buffers转化为目标数据类型。

————————————————————————

数据格式函数

> float() / double() / half() / bfloat16() 

将所有的parameters和buffers转化为指定的数据类型。

> to_empty()

把模型parameter和buffers移动到指定device上（不保存其具体数值）。